In [1]:
import os
import random
from tqdm import tqdm
import pandas as pd
import numpy as np
from glob import glob
import gc
from collections import defaultdict
import time

import cv2
import torch
import torch.nn as nn
from torch.nn.parallel import DataParallel
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
from torch.cuda import amp
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed(42)

# DataLoader

In [3]:
data_dir = \
    '/kaggle/input/blood-vessel-segmentation'

In [4]:
def file_to_id(f):
    s = f.split('/')
    return s[-3]+'_' + s[-1][:-4]

valid_meta = []
valid_folder = sorted(glob(f'{data_dir}/test/*'))
for image_folder in valid_folder:
    file = sorted(glob(f'{image_folder}/images/*.tif'))
    H, W = cv2.imread(file[0], cv2.IMREAD_ANYDEPTH).shape
    valid_meta.append(dict({
        'name':image_folder,
        'file':file,
        'shape':(len(file), H, W),
        'id':[file_to_id(f) for f in file],
    }))
        
        
print('len(valid_file) :', len(valid_meta))

len(valid_file) : 2


# Model1

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from timm.models import create_model


class sSE(nn.Module):
    def __init__(self, out_channels):
        super(sSE, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(out_channels, 1, kernel_size=1,padding=0),
                                  nn.BatchNorm2d(1),
                                  nn.Sigmoid())
    def forward(self,x):
        x=self.conv(x)
        return x

class cSE(nn.Module):
    def __init__(self, out_channels):
        super(cSE, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(out_channels, int(out_channels/2), kernel_size=1,padding=0),
                                   nn.BatchNorm2d(int(out_channels/2)),
                                   nn.ReLU(inplace=True),
                                   )
                        
        self.conv2 = nn.Sequential(nn.Conv2d(int(out_channels/2), out_channels, kernel_size=1,padding=0),
                                   nn.BatchNorm2d(out_channels),
                                   nn.Sigmoid(),
                                   )
    def forward(self,x):
        x=nn.AvgPool2d(x.size()[2:])(x)
        x=self.conv1(x)
        x=self.conv2(x)
        return x
    
class MyDecoderBlock(nn.Module):
    def __init__(
        self,
        in_channel,
        skip_channel,
        out_channel,
    ):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channel + skip_channel, out_channel, kernel_size=3, padding=1,),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channel,out_channel,kernel_size=3, padding=1,),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
        )
        self.spatial_gate = sSE(out_channel)
        self.channel_gate = cSE(out_channel)


    def forward(self, x, skip=None):
        x = F.interpolate(x, scale_factor=2, mode='bilinear')
        if skip is not None:
            x = torch.cat([x, skip], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        g1 = self.spatial_gate(x)
        g2 = self.channel_gate(x)
        x = g1*x + g2*x
        return x

class MyUnetDecoder(nn.Module):
    def __init__(self,
                 in_channel,
                 skip_channel,
                 out_channel,
                 ):
        super().__init__()

        self.center = nn.Identity()

        i_channel = [in_channel, ] + out_channel[:-1]
        s_channel = skip_channel
        o_channel = out_channel
        block = [
            MyDecoderBlock(i, s, o,)
            for i, s, o in zip(i_channel, s_channel, o_channel)
        ]
        self.block = nn.ModuleList(block)

    def forward(self, feature, skip):
        d = self.center(feature)

        for i, block in enumerate(self.block):
            s = skip[i]
            d = block(d, s)
            
        last = d
        return last


class ConvNeXt_U(nn.Module):
    def __init__(self):
        super().__init__() 
        encoder_dim = [24, 48, 96, 192, 384, 768]
        decoder_dim = [384, 192, 96, 48, 24]

        self.encoder = create_model('convnext_small.fb_in22k', pretrained=False, in_chans=1)

        self.decoder = MyUnetDecoder(
            in_channel  = encoder_dim[-1],
            skip_channel= encoder_dim[:-1][::-1],
            out_channel = decoder_dim,
        )
        self.vessel = nn.Conv2d(decoder_dim[-1], 1, kernel_size=1)
        self.kidney = nn.Conv2d(decoder_dim[-1], 1, kernel_size=1)
        self.stem0 = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=24, kernel_size=3, stride=1, padding=1), 
                                   nn.BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=24, out_channels=24, kernel_size=3, stride=1, padding=1), 
                                   nn.BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 
                                   nn.ReLU(inplace=True),
                                  )
        self.stem1 = nn.Sequential(nn.Conv2d(in_channels=24, out_channels=48, kernel_size=3, stride=1, padding=1), 
                                   nn.BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=48, out_channels=48, kernel_size=3, stride=1, padding=1), 
                                   nn.BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 
                                   nn.ReLU(inplace=True),
                                  )

    def forward(self, image):
        B, C, H, W = image.shape
        H_pad = (32 - H % 32) % 32
        W_pad = (32 - W % 32) % 32
        x = F.pad(image, (0, W_pad, 0, H_pad), 'constant', 0)
        # x = x.expand(-1, 3, -1, -1)

        encode = []
        xx = self.stem0(x); encode.append(xx)
        xx = F.avg_pool2d(xx,kernel_size=2,stride=2)
        xx = self.stem1(xx); encode.append(xx)

        e = self.encoder
        x = e.stem(x);

        x = e.stages[0](x); encode.append(x)
        x = e.stages[1](x); encode.append(x)
        x = e.stages[2](x); encode.append(x)
        x = e.stages[3](x); encode.append(x)
        #[print(f'encode_{i}', e.shape) for i,e in enumerate(encode)]
        last = self.decoder(
            feature=encode[-1], skip=encode[:-1][::-1]
        )

        vessel = self.vessel(last).float()
        vessel = F.logsigmoid(vessel).exp()
        vessel = vessel[:, :, :H, :W].contiguous()
        
        kidney = self.kidney(last).float()
        kidney = F.logsigmoid(kidney).exp()
        kidney = kidney[:, :, :H, :W].contiguous()
        
        return vessel, kidney
    
def run_check_net():
    height, width = 260, 256
    batch_size = 2

    image = torch.from_numpy(np.random.uniform(0, 1, (batch_size, 1, height, width))).float().to(device)

    net = ConvNeXt_U().to(device)

    with torch.no_grad():
        with torch.cuda.amp.autocast(enabled=True):
            v, k = net(image)

    print('image', image.shape)
    print('vessel', v.shape)
    print('kidney', k.shape)

if __name__ == '__main__':
    run_check_net()
    torch.cuda.empty_cache()

image torch.Size([2, 1, 260, 256])
vessel torch.Size([2, 1, 260, 256])
kidney torch.Size([2, 1, 260, 256])


In [6]:
model1 = ConvNeXt_U()
model1.load_state_dict(torch.load("/kaggle/input/sennet-hoa-models/convnext_small-unet-1ltuf51v.pt", map_location = device))
model1.eval()

ConvNeXt_U(
  (encoder): ConvNeXt(
    (stem): Sequential(
      (0): Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (stages): Sequential(
      (0): ConvNeXtStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): ConvNeXtBlock(
            (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=96, out_features=384, bias=True)
              (act): GELU()
              (drop1): Dropout(p=0.0, inplace=False)
              (norm): Identity()
              (fc2): Linear(in_features=384, out_features=96, bias=True)
              (drop2): Dropout(p=0.0, inplace=False)
            )
            (shortcut): Identity()
            (drop_path): Identity()
          )
          (1): ConvNeXtBlock(
            (conv_dw): Conv

In [7]:
model1 = model1.to(device)

# Model2

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from timm.models import create_model


class sSE(nn.Module):
    def __init__(self, out_channels):
        super(sSE, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(out_channels, 1, kernel_size=1,padding=0),
                                  nn.BatchNorm2d(1),
                                  nn.Sigmoid())
    def forward(self,x):
        x=self.conv(x)
        return x

class cSE(nn.Module):
    def __init__(self, out_channels):
        super(cSE, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(out_channels, int(out_channels/2), kernel_size=1,padding=0),
                                   nn.BatchNorm2d(int(out_channels/2)),
                                   nn.ReLU(inplace=True),
                                   )
                        
        self.conv2 = nn.Sequential(nn.Conv2d(int(out_channels/2), out_channels, kernel_size=1,padding=0),
                                   nn.BatchNorm2d(out_channels),
                                   nn.Sigmoid(),
                                   )
    def forward(self,x):
        x=nn.AvgPool2d(x.size()[2:])(x)
        x=self.conv1(x)
        x=self.conv2(x)
        return x
    
class MyDecoderBlock(nn.Module):
    def __init__(
        self,
        in_channel,
        skip_channel,
        out_channel,
    ):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channel + skip_channel, out_channel, kernel_size=3, padding=1,),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channel,out_channel,kernel_size=3, padding=1,),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
        )
        self.spatial_gate = sSE(out_channel)
        self.channel_gate = cSE(out_channel)


    def forward(self, x, skip=None):
        x = F.interpolate(x, scale_factor=2, mode='bilinear')
        if skip is not None:
            x = torch.cat([x, skip], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        g1 = self.spatial_gate(x)
        g2 = self.channel_gate(x)
        x = g1*x + g2*x
        return x

class MyUnetDecoder(nn.Module):
    def __init__(self,
                 in_channel,
                 skip_channel,
                 out_channel,
                 ):
        super().__init__()

        self.center = nn.Identity()

        i_channel = [in_channel, ] + out_channel[:-1]
        s_channel = skip_channel
        o_channel = out_channel
        block = [
            MyDecoderBlock(i, s, o,)
            for i, s, o in zip(i_channel, s_channel, o_channel)
        ]
        self.block = nn.ModuleList(block)

    def forward(self, feature, skip):
        d = self.center(feature)

        for i, block in enumerate(self.block):
            s = skip[i]
            d = block(d, s)
            
        last = d
        return last


class ConvNeXt_U(nn.Module):
    def __init__(self):
        super().__init__() 
        encoder_dim = [24, 48, 96, 192, 384, 768]
        decoder_dim = [384, 192, 96, 48, 24]

        self.encoder = create_model('convnext_small.fb_in22k', pretrained=False, in_chans=3)

        self.decoder = MyUnetDecoder(
            in_channel  = encoder_dim[-1],
            skip_channel= encoder_dim[:-1][::-1],
            out_channel = decoder_dim,
        )
        self.vessel = nn.Conv2d(decoder_dim[-1], 1, kernel_size=1)
        self.kidney = nn.Conv2d(decoder_dim[-1], 1, kernel_size=1)
        self.stem0 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=24, kernel_size=3, stride=1, padding=1), 
                                   nn.BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=24, out_channels=24, kernel_size=3, stride=1, padding=1), 
                                   nn.BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 
                                   nn.ReLU(inplace=True),
                                  )
        self.stem1 = nn.Sequential(nn.Conv2d(in_channels=24, out_channels=48, kernel_size=3, stride=1, padding=1), 
                                   nn.BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(in_channels=48, out_channels=48, kernel_size=3, stride=1, padding=1), 
                                   nn.BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 
                                   nn.ReLU(inplace=True),
                                  )

    def forward(self, image):
        B, C, H, W = image.shape
        H_pad = (32 - H % 32) % 32
        W_pad = (32 - W % 32) % 32
        x = F.pad(image, (0, W_pad, 0, H_pad), 'constant', 0)
        x = x.expand(-1, 3, -1, -1)

        encode = []
        xx = self.stem0(x); encode.append(xx)
        xx = F.avg_pool2d(xx,kernel_size=2,stride=2)
        xx = self.stem1(xx); encode.append(xx)

        e = self.encoder
        x = e.stem(x);

        x = e.stages[0](x); encode.append(x)
        x = e.stages[1](x); encode.append(x)
        x = e.stages[2](x); encode.append(x)
        x = e.stages[3](x); encode.append(x)
        #[print(f'encode_{i}', e.shape) for i,e in enumerate(encode)]
        last = self.decoder(
            feature=encode[-1], skip=encode[:-1][::-1]
        )

        vessel = self.vessel(last).float()
        vessel = F.logsigmoid(vessel).exp()
        vessel = vessel[:, :, :H, :W].contiguous()
        
        kidney = self.kidney(last).float()
        kidney = F.logsigmoid(kidney).exp()
        kidney = kidney[:, :, :H, :W].contiguous()
        
        return vessel, kidney
    
def run_check_net():
    height, width = 260, 256
    batch_size = 2

    image = torch.from_numpy(np.random.uniform(0, 1, (batch_size, 1, height, width))).float().to(device)

    net = ConvNeXt_U().to(device)

    with torch.no_grad():
        with torch.cuda.amp.autocast(enabled=True):
            v, k = net(image)

    print('image', image.shape)
    print('vessel', v.shape)
    print('kidney', k.shape)

if __name__ == '__main__':
    run_check_net()
    torch.cuda.empty_cache()

image torch.Size([2, 1, 260, 256])
vessel torch.Size([2, 1, 260, 256])
kidney torch.Size([2, 1, 260, 256])


In [9]:
model2 = ConvNeXt_U()
model2.load_state_dict(torch.load("/kaggle/input/sennet-hoa-models/convnext_small-unet-1cbx0ckf.pt", map_location = device))
model2.eval()
model2 = model2.to(device)

# Model3

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from timm.models import create_model


class MyDecoderBlock(nn.Module):
    def __init__(
        self,
        in_channel,
        skip_channel,
        out_channel,
    ):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channel + skip_channel, out_channel, kernel_size=3, padding=1,),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channel,out_channel,kernel_size=3, padding=1,),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
        )


    def forward(self, x, skip=None):
        x = F.interpolate(x, scale_factor=2, mode='bilinear')
        if skip is not None:
            x = torch.cat([x, skip], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        return x


class MyUnetDecoder(nn.Module):
    def __init__(self,
                 in_channel,
                 skip_channel,
                 out_channel,
                 ):
        super().__init__()

        self.center = nn.Identity()

        i_channel = [in_channel, ] + out_channel[:-1]
        s_channel = skip_channel
        o_channel = out_channel
        block = [
            MyDecoderBlock(i, s, o,)
            for i, s, o in zip(i_channel, s_channel, o_channel)
        ]
        self.block = nn.ModuleList(block)

    def forward(self, feature, skip):
        d = self.center(feature)

        for i, block in enumerate(self.block):
            s = skip[i]
            d = block(d, s)
            
        last = d
        return last


class ConvNeXt_U(nn.Module):
    def __init__(self):
        super().__init__() 
        encoder_dim = [24, 48, 96, 192, 384, 768]
        decoder_dim = [384, 192, 96, 48, 24]

        self.encoder = create_model('convnext_small.fb_in22k', pretrained=False, in_chans=3)

        self.decoder = MyUnetDecoder(
            in_channel  = encoder_dim[-1],
            skip_channel= encoder_dim[:-1][::-1],
            out_channel = decoder_dim,
        )
        self.vessel = nn.Conv2d(decoder_dim[-1], 1, kernel_size=1)
        self.kidney = nn.Conv2d(decoder_dim[-1], 1, kernel_size=1)
        self.stem0 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=24, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), nn.ReLU(inplace=True))
        self.stem1 = nn.Sequential(nn.Conv2d(in_channels=24, out_channels=48, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), nn.ReLU(inplace=True))

    def forward(self, image):
        B, C, H, W = image.shape
        H_pad = (32 - H % 32) % 32
        W_pad = (32 - W % 32) % 32
        x = F.pad(image, (0, W_pad, 0, H_pad), 'constant', 0)
        x = x.expand(-1, 3, -1, -1)

        encode = []
        xx = self.stem0(x); encode.append(xx)
        xx = F.avg_pool2d(xx,kernel_size=2,stride=2)
        xx = self.stem1(xx); encode.append(xx)

        e = self.encoder
        x = e.stem(x);

        x = e.stages[0](x); encode.append(x)
        x = e.stages[1](x); encode.append(x)
        x = e.stages[2](x); encode.append(x)
        x = e.stages[3](x); encode.append(x)
        #[print(f'encode_{i}', e.shape) for i,e in enumerate(encode)]
        last = self.decoder(
            feature=encode[-1], skip=encode[:-1][::-1]
        )

        vessel = self.vessel(last).float()
        vessel = F.logsigmoid(vessel).exp()
        vessel = vessel[:, :, :H, :W].contiguous()
        
        kidney = self.kidney(last).float()
        kidney = F.logsigmoid(kidney).exp()
        kidney = kidney[:, :, :H, :W].contiguous()
        
        return vessel, kidney

In [11]:
model3 = ConvNeXt_U()
model3.load_state_dict(torch.load("/kaggle/input/sennet-hoa-models/convnext_small-unet-16ylc1b5.pt", map_location = device))
model3.eval()
model3 = model3.to(device)

# Model Cody

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from timm.models import create_model

class sSE(nn.Module):
    def __init__(self, out_channels):
        super(sSE, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(out_channels, 1, kernel_size=1,padding=0),
                                  nn.BatchNorm2d(1),
                                  nn.Sigmoid())
    def forward(self,x):
        x=self.conv(x)
        return x

class cSE(nn.Module):
    def __init__(self, out_channels):
        super(cSE, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(out_channels, int(out_channels/2), kernel_size=1,padding=0),
                                   nn.BatchNorm2d(int(out_channels/2)),
                                   nn.ReLU(inplace=True),
                                   )
                        
        self.conv2 = nn.Sequential(nn.Conv2d(int(out_channels/2), out_channels, kernel_size=1,padding=0),
                                   nn.BatchNorm2d(out_channels),
                                   nn.Sigmoid(),
                                   )
    def forward(self,x):
        x=nn.AvgPool2d(x.size()[2:])(x)
        x=self.conv1(x)
        x=self.conv2(x)
        return x
    
class MyDecoderBlock(nn.Module):
    def __init__(
        self,
        in_channel,
        skip_channel,
        out_channel,
    ):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channel + skip_channel, out_channel, kernel_size=3, padding=1,),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channel,out_channel,kernel_size=3, padding=1,),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
        )
#         self.spatial_gate = sSE(out_channel)
#         self.channel_gate = cSE(out_channel)


    def forward(self, x, skip=None):
        x = F.interpolate(x, scale_factor=2, mode='bilinear')
        if skip is not None:
            x = torch.cat([x, skip], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
#         g1 = self.spatial_gate(x)
#         g2 = self.channel_gate(x)
#         x = g1*x + g2*x
        return x

class MyUnetDecoder(nn.Module):
    def __init__(self,
                 in_channel,
                 skip_channel,
                 out_channel,
                 ):
        super().__init__()

        self.center = nn.Identity()

        i_channel = [in_channel, ] + out_channel[:-1]
        s_channel = skip_channel
        o_channel = out_channel
        block = [
            MyDecoderBlock(i, s, o,)
            for i, s, o in zip(i_channel, s_channel, o_channel)
        ]
        self.block = nn.ModuleList(block)

    def forward(self, feature, skip):
        d = self.center(feature)

        for i, block in enumerate(self.block):
            s = skip[i]
            d = block(d, s)
            
        last = d
        return last
    
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from timm.models import create_model
class ConvNeXt_U(nn.Module):
    def __init__(self):
        super().__init__() 
        encoder_dim = [24, 48, 96, 192, 384, 768]
        decoder_dim = [384, 192, 96, 48, 24]

        self.encoder = create_model('convnext_small.fb_in22k', pretrained=False, in_chans=3)

        self.decoder = MyUnetDecoder(
            in_channel  = encoder_dim[-1],
            skip_channel= encoder_dim[:-1][::-1],
            out_channel = decoder_dim,
        )
        self.vessel = nn.Conv2d(decoder_dim[-1], 1, kernel_size=1)
        self.kidney = nn.Conv2d(decoder_dim[-1], 1, kernel_size=1)
        self.stem0 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=24, kernel_size=3, stride=1, padding=1), 
                                   nn.BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 
                                   nn.ReLU(inplace=True),
                                   )
        self.stem1 = nn.Sequential(nn.Conv2d(in_channels=24, out_channels=48, kernel_size=3, stride=1, padding=1), 
                                   nn.BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 
                                   nn.ReLU(inplace=True),
                                   )

    def forward(self, image):
        _, _, H, W = image.shape
        H_pad = (32 - H % 32) % 32
        W_pad = (32 - W % 32) % 32
        x = F.pad(image, (0, W_pad, 0, H_pad), 'constant', 0)
        x = x.expand(-1, 3, -1, -1)

        encode = []
        xx = self.stem0(x); encode.append(xx)
        xx = F.avg_pool2d(xx,kernel_size=2,stride=2)
        xx = self.stem1(xx); encode.append(xx)

        e = self.encoder
        x = e.stem(x);

        x = e.stages[0](x); encode.append(x)
        x = e.stages[1](x); encode.append(x)
        x = e.stages[2](x); encode.append(x)
        x = e.stages[3](x); encode.append(x)
        #[print(f'encode_{i}', e.shape) for i,e in enumerate(encode)]
        last = self.decoder(
            feature=encode[-1], skip=encode[:-1][::-1]
        )

        vessel = self.vessel(last).float()
        vessel = F.logsigmoid(vessel).exp()
        vessel = vessel[:, :, :H, :W].contiguous()
        
        kidney = self.kidney(last).float()
        kidney = F.logsigmoid(kidney).exp()
        kidney = kidney[:, :, :H, :W].contiguous()
        
        return vessel, kidney
    
def run_check_net():
    height, width = 260, 256
    batch_size = 2

    image = torch.from_numpy(np.random.uniform(0, 1, (batch_size, 1, height, width))).float().cuda()

    net = ConvNeXt_U().cuda()

    with torch.no_grad():
        with torch.cuda.amp.autocast(enabled=True):
            v, k = net(image)

    print('image', image.shape)
    print('vessel', v.shape)
    print('kidney', k.shape)

if __name__ == '__main__':
    run_check_net()
    torch.cuda.empty_cache()

image torch.Size([2, 1, 260, 256])
vessel torch.Size([2, 1, 260, 256])
kidney torch.Size([2, 1, 260, 256])


In [13]:
model_cody = ConvNeXt_U()

state_dict = torch.load("/kaggle/input/cody-convnext/convnext-unet-6fr2vqem.pt")
model_dict = model_cody.state_dict()
filtered_state_dict = {k: v for k, v in state_dict.items() if k in model_dict}
model_cody.load_state_dict(filtered_state_dict)

model_cody.eval()
model_cody = model_cody.to(device)

In [14]:
def rle_encode(mask):
    pixel = mask.flatten()
    pixel = np.concatenate([[0], pixel, [0]])
    run = np.where(pixel[1:] != pixel[:-1])[0] + 1
    run[1::2] -= run[::2]
    rle = ' '.join(str(r) for r in run)
    if rle == '':
        rle = '1 0'
    return rle

In [15]:
def make_prediction(predict, model, volume, shape, axes):
    import torchvision.transforms.functional as F
    model = DataParallel(model)
    for axis in axes:
        loader = np.array_split(np.arange((D, H, W)[axis]), max(1, int((D, H, W)[axis] // 8)))
        num_valid = len(loader)

        B = 0 
        for t in range(num_valid):

            if axis == 0:
                image = volume[loader[t].tolist()]
            if axis == 1:
                image = volume[:, loader[t].tolist()]
                image = image.transpose(1, 0, 2)
            if axis == 2:
                image = volume[:, :, loader[t].tolist()]
                image = image.transpose(2, 0, 1)

            batch_size, bh, bw = image.shape
            
            images = np.zeros_like(image)

            for i in range(batch_size):
                img = image[i]
                img = img.reshape(-1)
                img = (img - img.min()) / (img.max() - img.min() + 0.0001)
                img = img.reshape(bh, bw)
                img = np.ascontiguousarray(img)
                images[i] = img
            image = torch.from_numpy(images).float().to(device).unsqueeze(1)

            counter = 0
            vessel1, kidney1 = 0, 0
            vessel2, kidney2 = 0, 0
            image = image.to(device)
            with torch.cuda.amp.autocast():
                with torch.no_grad():
                    augmented_image = F.adjust_brightness(image, 1.1)
                    augmented_image = F.adjust_contrast(augmented_image, 1.1)
                    v1, k1 = model(augmented_image)
                    vessel1 += v1
                    kidney1 += k1
                    counter += 1
                    
                    v1, k1 = model(image)
                    vessel1 += v1
                    kidney1 += k1
                    counter += 1

                    v1, k1 = model(torch.flip(image, dims=[2,]))
                    vessel1 += torch.flip(v1, dims=[2,])
                    kidney1 += torch.flip(k1, dims=[2,])
                    counter += 1

                    v1, k1 = model(torch.flip(image, dims=[3,]))
                    vessel1 += torch.flip(v1, dims=[3,])
                    kidney1 += torch.flip(k1, dims=[3,])
                    counter += 1

                    flipped_image = torch.flip(image, dims=[2, 3])
                    v1, k1 = model(flipped_image)
                    vessel1 += torch.flip(v1, dims=[2, 3])
                    kidney1 += torch.flip(k1, dims=[2, 3])
                    counter += 1

            vessel1 = vessel1/counter   
            kidney1 = kidney1/counter         

            vessel1 = vessel1.float().data.cpu().numpy()
            kidney1 = kidney1.float().data.cpu().numpy()
            kidney1 = kidney1 > 0.5

            batch_size = len(vessel1)
            for b in range(batch_size):
                mk1 = kidney1[b, 0]
                mv1 = vessel1[b, 0]

                p1 = (mv1 * mk1)
                if axis == 0:
                    predict[B + b] += (p1/4)
                if axis == 1:
                    predict[:, B + b] += (p1/4)
                if axis == 2:
                    predict[:, :, B + b] += (p1/4)
            B += batch_size
        
    del model

In [16]:
start = time.time()
submission_df = []
for d in valid_meta:
    predict = np.zeros(d['shape'], dtype=np.float32)
    volume = [cv2.imread(f, cv2.IMREAD_ANYDEPTH).astype(np.float32) for f in d['file']]
    volume = np.stack(volume)
    D, H, W = volume.shape

    axes = [0,1,2]
    
    make_prediction(predict, model1, volume, (D, H, W), axes)
    torch.cuda.empty_cache()
    gc.collect()
    
    make_prediction(predict, model2, volume, (D, H, W), axes)
    torch.cuda.empty_cache()
    gc.collect()
    
    make_prediction(predict, model_cody, volume, (D, H, W), axes)
    torch.cuda.empty_cache()
    gc.collect()
    
    make_prediction(predict, model3, volume, (D, H, W), axes)
    torch.cuda.empty_cache()
    gc.collect()

    predict = (predict > 0.68).astype(np.uint8)
    
    rle = [rle_encode(p) for p in predict]
    
    submission_df.append(
            pd.DataFrame(data={
                'id'  : d['id'],
                'rle' : rle,
            })
        )
    
    del volume, predict, rle
    gc.collect()
    torch.cuda.empty_cache()
print(time.time() - start)

942.0633494853973


In [17]:
#cody model excluded: 0.869
#model 1 excluded:    0.872
#model 2 excluded:    0.865
#model 3 excluded:    0.871

In [18]:
submission = pd.concat(submission_df)
submission.to_csv('submission.csv', index=False)

In [19]:
submission.head()

,id,rle
0,kidney_5_0000,1 0
1,kidney_5_0001,1 0
2,kidney_5_0002,1 0
0,kidney_6_0000,1 0
1,kidney_6_0001,1 0
